In [1]:
# for reference see this code: /home/csutter/DRIVE/weather_img_concatmodels/cnn_hrrr_fcsthr2/nested_skill_streamline_vs_blending.ipynb

import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", None)        # Use full width of the console
pd.set_option("display.max_colwidth", None)  # Show full content in each column

# CNN model selection

This file: /home/csutter/DRIVE-clean/CNN/data_results/results_by_exp_innerVal.csv 
- Has Baseline model performance appended for each of the 36 BL models (hyperparams L2 and dropout set to 0)
- Has HT model performance appended for each of the 64 HT models (16 hyperparam sets x top 4 BL models)
- For final model selection, consider all 36 (with initial HTs of 0s) + 64 (with other HTs tuned) for final selection - this is why all models were appended into the same csv file.

### CNN - Baseline selection (BL)

In [2]:
##### For Baseline top 4 model selections

df = pd.read_csv("/home/csutter/DRIVE-clean/CNN/data_results/results_by_exp_innerVal.csv")

# Important! If recreating the selection from the 36 baseline models, run this code below to subset just the first 36 runs. O/w if doing final selection post hyptuning, don't need to subset - want to consider all models. See markdown note above for more detail.
df = df.head(36)

df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)


top10_avgrecall = df.sort_values("avg_recall", ascending=False)[0:10]
selected4 = top10_avgrecall.sort_values("overall", ascending=False)[0:4]

display(selected4)

print(selected4["exp_details"])

,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
20,107445,74272,7635,5651,14945,8968,42290,31316,38000,25133,4575,3204,97697,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue,2025-07-01 13:22:20.463990,0.691256,0.661395,0.700328,0.600067,0.740144,0.740506,0.688488
2,107445,73410,7635,5625,14945,8582,42290,30997,38000,25063,4575,3143,97679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse,2025-06-29 03:06:33.552986,0.683233,0.659553,0.686995,0.574239,0.736739,0.732963,0.678098
26,107445,73123,7635,5655,14945,8844,42290,30146,38000,25309,4575,3169,97397,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_DR0_E75_AugTrue,2025-07-01 14:03:48.795248,0.680562,0.666026,0.692678,0.591770,0.740668,0.712840,0.680796
4,107445,72893,7635,5761,14945,9210,42290,30253,38000,24391,4575,3278,96548,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse,2025-06-29 03:20:17.279312,0.678422,0.641868,0.716503,0.616260,0.754551,0.715370,0.688910


20        _A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue
2        _A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse
26         _A_resnet_TRLETrue_ASTTrue_L20_DR0_E75_AugTrue
4     _A_mobilenet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse
Name: exp_details, dtype: object


In [3]:
# # TESTING SOME OTHER VARIATIONS
# print("AVERAGE WAY")
# # df["avgmetrics"] = df[['overall', 'avg_recall']].mean(axis=1)
# df["avgmetrics"] = 0.25*df['overall']+0.75*df['avg_recall']
# avgmetrics = df.sort_values("avgmetrics", ascending=False)[0:10]
# display(avgmetrics)

# print("IMPORTANCES")
# importances = [2,1.5,1,1,1.5]
# cols = ["perc_snow_severe", "perc_snow", "perc_wet", "perc_dry", "perc_poor_viz"]
# values = df[cols].values
# df["weighted_avg_recall"] = (values * importances).sum(axis=1) / sum(importances)
# wtdavg = df.sort_values("weighted_avg_recall", ascending=False)[0:10]
# display(wtdavg)

# print("IMPORTANCE + AVG")
# # df["avgmetrics"] = df[['overall', 'avg_recall']].mean(axis=1)
# df["impAndAvg"] = (1/3)*df['overall']+(1/3)*df['avg_recall']+(1/3)*df["weighted_avg_recall"] 
# impAndAvg = df.sort_values("impAndAvg", ascending=False)[0:10]
# display(impAndAvg)

### CNN - Hyperparameter tuning (HT) selection

In [7]:
##### for HT model selection

# for each of the 4 BL models that hyp tuning was performed, grab the best model from each, and move forward with HTing EACH of those for the + weather data. 

df = pd.read_csv("/home/csutter/DRIVE-clean/CNN/data_results/results_by_exp_innerVal.csv")

df["overall"] = df["correctims"]/df["totalims"]

for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)


# add col with the BL model type (i.e., the 4 BL models) from which we are then selecting the best one hyperparams
model_BL_desc = []
for i in df["exp_details"]:
    b = i.find("L2")
    e = i.rfind("Aug")
    desc = i[:b]+i[e:]
    model_BL_desc.append(desc)

df["model_BL_desc"] = model_BL_desc

unique_BL = np.unique(df["model_BL_desc"])


### The way I originally did it with selecting top one model PER each baseline model
for bl in unique_BL:
    dfsub = df[df["model_BL_desc"]==bl]
    if len(dfsub)>1: # only look at the selected 4 BL models, not all the other 32 with only the one BL run exp
        print(len(dfsub))
        top4_avgrecall = dfsub.sort_values("avg_recall", ascending=False)[0:5] # since only 16 HT are run, select from top 5 (roughly 1/3)
        selected1 = top4_avgrecall.sort_values("overall", ascending=False)[0:1] # grab the top 1
        print(bl)
        display(selected1)
        print(selected1["exp_details"])

17
_A_mobilenet_TRLETrue_ASTFalse_AugFalse


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc
57,107445,71893,7635,5911,14945,9531,42290,29259,38000,23703,4575,3489,95897,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse,2025-07-08 18:38:11.556300,0.669114,0.623763,0.762623,0.637738,0.774198,0.691866,0.698038,_A_mobilenet_TRLETrue_ASTFalse_AugFalse


57    _A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse
Name: exp_details, dtype: object
17
_A_resnet_TRLETrue_ASTFalse_AugFalse


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc
87,107445,72769,7635,5874,14945,9112,42290,29897,38000,24177,4575,3709,96736,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-14 02:57:04.840090,0.677267,0.636237,0.81071,0.609702,0.769352,0.706952,0.706591,_A_resnet_TRLETrue_ASTFalse_AugFalse


87    _A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse
Name: exp_details, dtype: object
17
_A_resnet_TRLETrue_ASTFalse_AugTrue


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc
74,107445,73460,7635,5787,14945,9243,42290,29897,38000,25025,4575,3508,97365,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue,2025-07-14 01:21:32.194943,0.683699,0.658553,0.766776,0.618468,0.757957,0.706952,0.701741,_A_resnet_TRLETrue_ASTFalse_AugTrue


74    _A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue
Name: exp_details, dtype: object
17
_A_resnet_TRLETrue_ASTTrue_AugTrue


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc
39,107445,74274,7635,5909,14945,9010,42290,30413,38000,25346,4575,3596,97484,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,2025-07-08 11:27:22.701127,0.691275,0.667,0.786011,0.602877,0.773936,0.719153,0.709795,_A_resnet_TRLETrue_ASTTrue_AugTrue


39    _A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue
Name: exp_details, dtype: object


In [4]:
# TESTING SOME OTHER VARIATIONS


# ### Trying where I select just the top ONE
# top4_avgrecall = df.sort_values("avg_recall", ascending=False)[0:10] # since only 16 HT are run, select from top 5 (roughly 1/3)
# selected1 = top4_avgrecall.sort_values("overall", ascending=False)[0:1] 
# display(selected1)


# # TESTING SOME OTHER VARIATIONS
# print("AVERAGE WAY")
# for bl in unique_BL:
#     dfsub = df[df["model_BL_desc"]==bl]
#     if len(dfsub)>1: # only look at the selected 4 BL models, not all the other 32 with only the one BL run exp
#         print(len(dfsub))
#         # dfsub["avgmetrics"] = dfsub[['overall', 'avg_recall']].mean(axis=1)
#         dfsub["avgmetrics"] = 0.5*dfsub['overall']+0.5*dfsub['avg_recall']
#         avgmetrics = dfsub.sort_values("avgmetrics", ascending=False)[0:1]
#         display(avgmetrics)# grab the top 1

# print("AVG OVERALL (not PER BL model)")
# df["avgmetrics"] = 0.5*df['overall']+0.5*df['avg_recall']
# avgmetrics = df.sort_values("avgmetrics", ascending=False)[0:10]
# display(avgmetrics)

# print("IMPORTANCES")
# importances = [2,1.5,1,1,1.5]
# cols = ["perc_snow_severe", "perc_snow", "perc_wet", "perc_dry", "perc_poor_viz"]
# values = df[cols].values
# df["weighted_avg_recall"] = (values * importances).sum(axis=1) / sum(importances)
# wtdavg = df.sort_values("weighted_avg_recall", ascending=False)[0:10]
# display(wtdavg)

# print("IMPORTANCE + AVG")
# # df["avgmetrics"] = df[['overall', 'avg_recall']].mean(axis=1)
# df["impAndAvg"] = (1/3)*df['overall']+(1/3)*df['avg_recall']+(1/3)*df["weighted_avg_recall"] 
# impAndAvg = df.sort_values("impAndAvg", ascending=False)[0:10]
# display(impAndAvg)

In [36]:
5/17

0.29411764705882354

In [39]:
10/36

0.2777777777777778

In [40]:
54960*.28

15388.800000000001

# Downstream model
- After models are run (see dir: /home/csutter/DRIVE-clean/downstream), results are appended to here: /home/csutter/DRIVE-clean/downstream/data_results/results_by_each_run_innerVal.csv
- First chunk below aggregates results by model since the runs were appended by fold, which we need to aggregate across.
- Then can select, similar to baseline/HT selection of CNN, to select best from that aggregated file -- last chunk of code

In [12]:
# Already ran this aggregation code, don't need to run again!

# Important first step: That csv has each model|fold innerVal results appended, but need to aggregate across the 30 folds to make selection of best model, so FIRST need to preprocess to aggregate at the model level, rather than model|fold level

results_model_fold = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_each_run_innerVal.csv")

results_model_fold.head(4)

aggregated = results_model_fold.groupby(["exp_desc","exp_details"]).sum().reset_index()
aggregated.head(5)

# Save this out so don't need to run again
aggregated.to_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

/tmp/tmp.uWXsIPpu6e/ipykernel_2774489/286293116.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aggregated = results_model_fold.groupby(["exp_desc","exp_details"]).sum().reset_index()


In [16]:
results_model_fold = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_each_run_innerVal.csv")

results_model_fold.head(4)

,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,datafold
0,3697,2814,294,178,511,297,1422,1154,1326,1086,144,99,3451,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,"logistic_{'max_iter': 100, 'C': 0.01}",nestcv_5cat_twotrain_OT1_m3_T5V0
1,3697,2800,294,177,511,304,1422,1140,1326,1086,144,93,3448,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,"logistic_{'max_iter': 100, 'C': 0.1}",nestcv_5cat_twotrain_OT1_m3_T5V0
2,3697,2789,294,175,511,308,1422,1122,1326,1100,144,84,3431,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,"logistic_{'max_iter': 100, 'C': 1.0}",nestcv_5cat_twotrain_OT1_m3_T5V0
3,3697,2791,294,177,511,307,1422,1123,1326,1101,144,83,3429,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,"logistic_{'max_iter': 100, 'C': 10.0}",nestcv_5cat_twotrain_OT1_m3_T5V0


### Downstream model selection

In [15]:
df = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]



df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

# grab one from  a specific arch
archs = []
for i in range(0,len(df)):
    desc = df["exp_desc"][i]
    desc = desc[2:]
    end = desc.find("_")
    a = desc[:end]
    archs.append(a)

df["arch"] = archs



# Decision: index 1507 showed up in a variety of different weighted-importances (when we tried these below), and replaced the initial top 10 with weighted recall rather than just plain average. Motivation to keep it simple and just move forward with the simplest -- top 10 avg recall, then take the max overall from that. Top models are all pretty similar, anyway. 
top10_avgrecall = df.sort_values("avg_recall", ascending=False)[0:10]
selected4 = top10_avgrecall.sort_values("overall", ascending=False)[0:10]

display(selected4)

print(selected4["exp_details"])
print(np.max(selected4["perc_snow"]))

,Unnamed: 0,exp_desc,exp_details,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,arch
1507,1507,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86690,7635,6114,14945,11190,42290,32202,38000,33864,4575,3320,102107,0.806831,0.891158,0.725683,0.748745,0.800786,0.761457,0.785566,resnet
1525,1525,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.75, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86638,7635,6093,14945,11206,42290,32149,38000,33882,4575,3308,102096,0.806347,0.891632,0.723060,0.749816,0.798035,0.760203,0.784549,resnet
1495,1495,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86567,7635,6109,14945,11120,42290,32194,38000,33804,4575,3340,102087,0.805687,0.889579,0.730055,0.744062,0.800131,0.761267,0.785019,resnet
1808,1808,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 1, 'gamma': 0.1}",107445,84035,7635,6287,14945,11159,42290,30537,38000,32490,4575,3562,100697,0.782121,0.855000,0.778579,0.746671,0.823445,0.722086,0.785156,resnet
1804,1804,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 1, 'gamma': 'scale'}",107445,83975,7635,6295,14945,11157,42290,30521,38000,32436,4575,3566,100614,0.781563,0.853579,0.779454,0.746537,0.824492,0.721707,0.785154,resnet
1815,1815,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 10, 'gamma': 0.01}",107445,83838,7635,6362,14945,11081,42290,30489,38000,32268,4575,3638,100234,0.780288,0.849158,0.795191,0.741452,0.833268,0.720951,0.788004,resnet
1358,1358,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,"svm_{'kernel': 'rbf', 'C': 10, 'gamma': 0.01}",107445,83658,7635,6320,14945,10999,42290,30616,38000,32102,4575,3621,100070,0.778612,0.844789,0.791475,0.735965,0.827767,0.723954,0.784790,resnet
901,901,A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 10, 'gamma': 0.01}",107445,83497,7635,6380,14945,10956,42290,30429,38000,32115,4575,3617,100104,0.777114,0.845132,0.790601,0.733088,0.835625,0.719532,0.784796,resnet
1822,1822,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 100, 'gamma': 0.001}",107445,83426,7635,6334,14945,11059,42290,30763,38000,31631,4575,3639,99739,0.776453,0.832395,0.795410,0.739980,0.829601,0.727430,0.784963,resnet
1807,1807,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 1, 'gamma': 0.01}",107445,83151,7635,6415,14945,11094,42290,30862,38000,31120,4575,3660,99573,0.773894,0.818947,0.800000,0.742322,0.840210,0.729771,0.786250,resnet


1507     rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}
1525    rf_{'max_depth': 10, 'max_samples': 0.75, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}
1495     rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}
1808                                                                                    svm_{'kernel': 'rbf', 'C': 1, 'gamma': 0.1}
1804                                                                                svm_{'kernel': 'rbf', 'C': 1, 'gamma': 'scale'}
1815                                                                                  svm_{'kernel': 'rbf', 'C': 10, 'gamma': 0.01}
1358                                                                                  svm_{'kernel': 'rbf', 'C': 10, 'gamma': 0.01}
901                                                                         

In [19]:
# TESTING SOME OTHER VARIATIONS

importances = [2,1.5,1,1,1.5]

cols = ["perc_snow_severe", "perc_snow", "perc_wet", "perc_dry", "perc_poor_viz"]
values = df[cols].values
df["weighted_avg_recall"] = (values * importances).sum(axis=1) / sum(importances)

print("AVERAGE WAY")
# df["avgmetrics"] = df[['overall', 'avg_recall']].mean(axis=1)
df["avgmetrics"] = 0.5*df['overall']+0.5*df['avg_recall']

avgmetrics = df.sort_values("avgmetrics", ascending=False)[0:10]
display(avgmetrics)

print("IMPORTANCES WAY")
top10_importance = df.sort_values("weighted_avg_recall", ascending=False)[30:50] 
display(top10_importance)
# just for testing. Tried: 
# importances = [1,2,2,3,1.5]
# importances = [0.9,1.2,1.2,1.5,1]

print("IMPORTANCE + AVG")
# df["avgmetrics"] = df[['overall', 'avg_recall']].mean(axis=1)
df["impAndAvg"] = (1/3)*df['overall']+(1/3)*df['avg_recall']+(1/3)*df["weighted_avg_recall"] 
impAndAvg = df.sort_values("impAndAvg", ascending=False)[0:10]
display(impAndAvg)


# grab example from specific arch
print("grab one arch specific")
exarch = df[df["arch"]=="mobilenet"].sort_values("avg_recall", ascending=False)[0:10]
# display(exarch)
top10_avgrecall = exarch.sort_values("avg_recall", ascending=False)[0:10]
selected4 = top10_avgrecall.sort_values("overall", ascending=False)[0:10]
display(selected4)

AVERAGE WAY


,Unnamed: 0,exp_desc,exp_details,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,arch,weighted_avg_recall,avgmetrics,impAndAvg
1507,1507,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86690,7635,6114,14945,11190,42290,32202,38000,33864,4575,3320,102107,0.806831,0.891158,0.725683,0.748745,0.800786,0.761457,0.785566,resnet,0.780833,0.796199,0.791077
1723,1723,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86791,7635,6113,14945,11022,42290,32671,38000,33664,4575,3321,102212,0.807771,0.885895,0.725902,0.737504,0.800655,0.772547,0.784500,resnet,0.779266,0.796136,0.790513
1711,1711,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_samples': 0.5, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86737,7635,6101,14945,11056,42290,32595,38000,33660,4575,3325,102212,0.807269,0.885789,0.726776,0.739779,0.799083,0.770750,0.784435,resnet,0.779220,0.795852,0.790308
1579,1579,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 20, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86736,7635,6105,14945,11027,42290,32694,38000,33587,4575,3323,102152,0.807260,0.883868,0.726339,0.737839,0.799607,0.773091,0.784149,resnet,0.778920,0.795704,0.790109
1525,1525,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.75, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86638,7635,6093,14945,11206,42290,32149,38000,33882,4575,3308,102096,0.806347,0.891632,0.723060,0.749816,0.798035,0.760203,0.784549,resnet,0.779603,0.795448,0.790167
1597,1597,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 20, 'max_samples': 0.75, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86839,7635,6069,14945,11051,42290,32754,38000,33683,4575,3282,102261,0.808218,0.886395,0.717377,0.739445,0.794892,0.774509,0.782524,resnet,0.776560,0.795371,0.789101
1495,1495,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86567,7635,6109,14945,11120,42290,32194,38000,33804,4575,3340,102087,0.805687,0.889579,0.730055,0.744062,0.800131,0.761267,0.785019,resnet,0.780326,0.795353,0.790344
1741,1741,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_samples': 0.75, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86802,7635,6070,14945,11031,42290,32775,38000,33650,4575,3276,102234,0.807874,0.885526,0.716066,0.738106,0.795023,0.775006,0.781945,resnet,0.775977,0.794910,0.788599
1543,1543,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 1.0, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86581,7635,6063,14945,11210,42290,32133,38000,33861,4575,3314,102095,0.805817,0.891079,0.724372,0.750084,0.794106,0.759825,0.783893,resnet,0.778686,0.794855,0.789465
1567,1567,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 20, 'max_samples': 0.5, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86682,7635,6108,14945,11034,42290,32669,38000,33576,4575,3295,102164,0.806757,0.883579,0.720219,0.738307,0.800000,0.772499,0.782921,resnet,0.777695,0.794839,0.789124


IMPORTANCES WAY


,Unnamed: 0,exp_desc,exp_details,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,arch,weighted_avg_recall,avgmetrics,impAndAvg
1453,1453,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"DNN_{'hidden_layers': 5, 'hidden_units': '[256, 128, 64, 32, 16]', 'dropout': 0.0, 'l2_reg': 0.0}",107445,83220,7635,6247,14945,11118,42290,30371,38000,31871,4575,3613,99919,0.774536,0.838711,0.789727,0.743928,0.818206,0.718160,0.781746,resnet,0.784823,0.778141,0.780368
1436,1436,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"DNN_{'hidden_layers': 4, 'hidden_units': '[128, 64, 32, 16]', 'dropout': 0.2, 'l2_reg': 0.01}",107445,81445,7635,6436,14945,10683,42290,29500,38000,31113,4575,3713,98915,0.758016,0.818763,0.811585,0.714821,0.842960,0.697564,0.777139,resnet,0.784551,0.767577,0.773235
1687,1687,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 5, 'max_samples': 1.0, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,84317,7635,6222,14945,11163,42290,29840,38000,33583,4575,3509,100669,0.784746,0.883763,0.766995,0.746939,0.814931,0.705604,0.783646,resnet,0.784304,0.784196,0.784232
908,908,A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue,"svm_{'kernel': 'rbf', 'C': 100, 'gamma': 0.001}",107445,83069,7635,6316,14945,10917,42290,30755,38000,31469,4575,3612,99630,0.773130,0.828132,0.789508,0.730478,0.827243,0.727240,0.780520,resnet,0.784263,0.776825,0.779304
1686,1686,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 5, 'max_samples': 1.0, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 1, 'bootstrap': True}",107445,84255,7635,6221,14945,11136,42290,29820,38000,33561,4575,3517,100676,0.784169,0.883184,0.768743,0.745132,0.814800,0.705131,0.783398,resnet,0.784104,0.783783,0.783890
1351,1351,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,"svm_{'kernel': 'rbf', 'C': 1, 'gamma': 0.1}",107445,83680,7635,6228,14945,11132,42290,30510,38000,32242,4575,3568,100475,0.778817,0.848474,0.779891,0.744865,0.815717,0.721447,0.782079,resnet,0.784070,0.780448,0.781655
998,998,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,"DNN_{'hidden_layers': 5, 'hidden_units': '[256, 128, 64, 32, 16]', 'dropout': 0.2, 'l2_reg': 0.001}",107445,81199,7635,6461,14945,10719,42290,29331,38000,30995,4575,3693,98458,0.755726,0.815658,0.807213,0.717230,0.846234,0.693568,0.775981,resnet,0.784051,0.765853,0.771919
1668,1668,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 5, 'max_samples': 0.75, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 1, 'bootstrap': True}",107445,84377,7635,6210,14945,11170,42290,29894,38000,33597,4575,3506,100711,0.785304,0.884132,0.766339,0.747407,0.813360,0.706881,0.783624,resnet,0.784050,0.784464,0.784326
1675,1675,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 5, 'max_samples': 1.0, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,84248,7635,6202,14945,11169,42290,29705,38000,33651,4575,3521,100652,0.784103,0.885553,0.769617,0.747340,0.812312,0.702412,0.783447,resnet,0.784004,0.783775,0.783851
521,521,A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue,"DNN_{'hidden_layers': 4, 'hidden_units': '[128, 64, 32, 16]', 'dropout': 0.2, 'l2_reg': 0.001}",107445,81602,7635,6402,14945,10859,42290,29553,38000,31118,4575,3670,98805,0.759477,0.818895,0.802186,0.726598,0.838507,0.698818,0.777001,resnet,0.783986,0.768239,0.773488


IMPORTANCE + AVG


,Unnamed: 0,exp_desc,exp_details,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,arch,weighted_avg_recall,avgmetrics,impAndAvg
1507,1507,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86690,7635,6114,14945,11190,42290,32202,38000,33864,4575,3320,102107,0.806831,0.891158,0.725683,0.748745,0.800786,0.761457,0.785566,resnet,0.780833,0.796199,0.791077
1723,1723,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86791,7635,6113,14945,11022,42290,32671,38000,33664,4575,3321,102212,0.807771,0.885895,0.725902,0.737504,0.800655,0.772547,0.784500,resnet,0.779266,0.796136,0.790513
1495,1495,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86567,7635,6109,14945,11120,42290,32194,38000,33804,4575,3340,102087,0.805687,0.889579,0.730055,0.744062,0.800131,0.761267,0.785019,resnet,0.780326,0.795353,0.790344
1711,1711,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_samples': 0.5, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86737,7635,6101,14945,11056,42290,32595,38000,33660,4575,3325,102212,0.807269,0.885789,0.726776,0.739779,0.799083,0.770750,0.784435,resnet,0.779220,0.795852,0.790308
1525,1525,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.75, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86638,7635,6093,14945,11206,42290,32149,38000,33882,4575,3308,102096,0.806347,0.891632,0.723060,0.749816,0.798035,0.760203,0.784549,resnet,0.779603,0.795448,0.790167
1579,1579,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 20, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86736,7635,6105,14945,11027,42290,32694,38000,33587,4575,3323,102152,0.807260,0.883868,0.726339,0.737839,0.799607,0.773091,0.784149,resnet,0.778920,0.795704,0.790109
1543,1543,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 1.0, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86581,7635,6063,14945,11210,42290,32133,38000,33861,4575,3314,102095,0.805817,0.891079,0.724372,0.750084,0.794106,0.759825,0.783893,resnet,0.778686,0.794855,0.789465
1050,1050,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86428,7635,6096,14945,11082,42290,32109,38000,33786,4575,3355,102029,0.804393,0.889105,0.733333,0.741519,0.798428,0.759258,0.784329,resnet,0.779643,0.794361,0.789455
1513,1513,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.75, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86516,7635,6091,14945,11161,42290,32137,38000,33818,4575,3309,102065,0.805212,0.889947,0.723279,0.746805,0.797773,0.759920,0.783545,resnet,0.778648,0.794378,0.789135
1567,1567,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 20, 'max_samples': 0.5, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86682,7635,6108,14945,11034,42290,32669,38000,33576,4575,3295,102164,0.806757,0.883579,0.720219,0.738307,0.800000,0.772499,0.782921,resnet,0.777695,0.794839,0.789124


grab one arch specific


,Unnamed: 0,exp_desc,exp_details,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,arch,weighted_avg_recall,avgmetrics,impAndAvg
196,196,A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse,"rf_{'max_depth': 20, 'max_samples': 0.5, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86445,7635,5869,14945,11140,42290,32836,38000,33449,4575,3151,101650,0.804551,0.880237,0.688743,0.745400,0.768697,0.776448,0.771905,mobilenet,0.763613,0.788228,0.780023
154,154,A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse,"rf_{'max_depth': 10, 'max_samples': 0.75, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86414,7635,5870,14945,11258,42290,32403,38000,33708,4575,3175,101641,0.804263,0.887053,0.693989,0.753295,0.768828,0.766210,0.773875,mobilenet,0.765978,0.789069,0.781372
160,160,A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse,"rf_{'max_depth': 10, 'max_samples': 1.0, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86394,7635,5852,14945,11284,42290,32351,38000,33749,4575,3158,101668,0.804077,0.888132,0.690273,0.755035,0.766470,0.764980,0.772978,mobilenet,0.764859,0.788527,0.780638
172,172,A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse,"rf_{'max_depth': 10, 'max_samples': 1.0, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86362,7635,5849,14945,11268,42290,32335,38000,33749,4575,3161,101614,0.803779,0.888132,0.690929,0.753965,0.766077,0.764602,0.772741,mobilenet,0.764604,0.788260,0.780374
136,136,A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86358,7635,5890,14945,11255,42290,32371,38000,33643,4575,3199,101571,0.803741,0.885342,0.699235,0.753095,0.771447,0.765453,0.774914,mobilenet,0.767455,0.789328,0.782037
142,142,A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse,"rf_{'max_depth': 10, 'max_samples': 0.75, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86313,7635,5880,14945,11287,42290,32350,38000,33644,4575,3152,101584,0.803323,0.885368,0.688962,0.755236,0.770138,0.764956,0.772932,mobilenet,0.765271,0.788127,0.780508
190,190,A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse,"rf_{'max_depth': 10, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': False}",107445,86310,7635,5861,14945,11319,42290,32233,38000,33740,4575,3157,101631,0.803295,0.887895,0.690055,0.757377,0.767649,0.762190,0.773033,mobilenet,0.765219,0.788164,0.780515
124,124,A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86292,7635,5893,14945,11226,42290,32374,38000,33602,4575,3197,101588,0.803127,0.884263,0.698798,0.751154,0.771840,0.765524,0.774316,mobilenet,0.766914,0.788722,0.781452
178,178,A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse,"rf_{'max_depth': 10, 'n_estimators': 100, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': False}",107445,86213,7635,5846,14945,11302,42290,32192,38000,33712,4575,3161,101557,0.802392,0.887158,0.690929,0.756240,0.765684,0.761220,0.772246,mobilenet,0.764357,0.787319,0.779665
297,297,A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse,"rf_{'max_depth': 5, 'max_samples': 0.75, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 1, 'bootstrap': True}",107445,83338,7635,5930,14945,11391,42290,29446,38000,33124,4575,3447,99996,0.775634,0.871684,0.753443,0.762195,0.776686,0.696288,0.772059,mobilenet,0.770686,0.773847,0.772793
